In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-bert-cv-def-wo-dknr

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  30.72kB
Step 1/32 : FROM nvidia/cuda:10.0-cudnn7-devel-ubuntu16.04
10.0-cudnn7-devel-ubuntu16.04: Pulling from nvidia/cuda
4007a89234b4: Pulling fs layer
c1de0f9cdfc1: Pulling fs layer
c8ee6ca703b8: Pulling fs layer
b39e2761d3d4: Pulling fs layer
5e86b4ffdb27: Pulling fs layer
8b0ca334b099: Pulling fs layer
8d58f2cfa4cc: Pulling fs layer
0d95951f5659: Pulling fs layer
2c172e6de5e4: Pulling fs layer
d98ead974717: Pulling fs layer
1312ad232c08: Pulling fs layer
c3150d340133: Pulling fs layer
3cfe1eec4dd3: Pulling fs layer
5e86b4ffdb27: Waiting
8b0ca334b099: Waiting
8d58f2cfa4cc: Waiting
0d95951f5659: Waiting
2c172e6de5e4: Waiting
d98ead974717: Waiting
1312ad232c08: Waiting
c3150d340133: Waiting
3cfe1eec4dd3: Waiting
b39e2761d3d4: Waiting
c8ee6ca703b8: Verifying Checksum
c8ee6ca703b8: Download complete
c1de0f9cdfc1: Verifying Checksum
c1de0f9cdfc1: Download complete
b39e2761d3d4: Verifying Checksum
b39e2761d3d4: Download complete
5e8

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 196 ms, sys: 35.7 ms, total: 232 ms
Wall time: 31min 13s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_bert_cv_def_wo_dknr'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-bert-cv-def-wo-dknr:latest'.format(account, region)
vdok3bert = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3bert.fit(data_location)

2021-05-15 02:56:27 Starting - Starting the training job...
2021-05-15 02:56:49 Starting - Launching requested ML instancesProfilerReport-1621047387: InProgress
......
2021-05-15 02:57:51 Starting - Preparing the instances for training......
2021-05-15 02:58:56 Downloading - Downloading input data...
2021-05-15 02:59:11 Training - Downloading the training image.....................
2021-05-15 03:02:52 Training - Training image download completed. Training in progress../.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Starting the training.
bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_ty

In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3bert.deploy(1, 'ml.p3.2xlarge', serializer=csv_serializer)

---------------------!

In [7]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Score_Class,Definition-Score
1,0
1,1
1,1
2,2



In [8]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3bert.transformer(instance_count=1,
                               instance_type='ml.p3.2xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

In [10]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

............................................Starting the inference server with 8 workers.
[2021-05-15 03:27:58 +0000] [41] [INFO] Starting gunicorn 20.1.0
[2021-05-15 03:27:58 +0000] [41] [INFO] Listening at: unix:/tmp/gunicorn.sock (41)
[2021-05-15 03:27:58 +0000] [41] [INFO] Using worker: gevent
[2021-05-15 03:27:58 +0000] [45] [INFO] Booting worker with pid: 45
[2021-05-15 03:27:58 +0000] [46] [INFO] Booting worker with pid: 46
[2021-05-15 03:27:58 +0000] [47] [INFO] Booting worker with pid: 47
[2021-05-15 03:27:58 +0000] [48] [INFO] Booting worker with pid: 48
[2021-05-15 03:27:58 +0000] [56] [INFO] Booting worker with pid: 56
[2021-05-15 03:27:58 +0000] [71] [INFO] Booting worker with pid: 71
[2021-05-15 03:27:58 +0000] [79] [INFO] Booting worker with pid: 79
[2021-05-15 03:27:58 +0000] [80] [INFO] Booting worker with pid: 80
/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float`